In [1]:
from GameVariants import PlayersGame
from Deck import Deck, Card
from players.CiroPlayer import CiroPlayer
from players.RandomPlayer import RandomPlayer
from players.LodoPlayer import LodoPlayer
from players.MonteCarloPlayer import MonteCarloPlayer
from PlayerNode import goto_player
import numpy as np
import math

# Analysis of game statistics (given random players)

the question to anser are:
- Who wins the most?
- Who loses the most?
- Is there a best strategy for predictions?
- Is there a best strategy for playing cards?
- Are the first or last player of a round in advantage?
- What is the right number to predict with n player and c cards?

## Check if there is a connection between winning and starting first or last

In [ ]:
class FixedGame(PlayersGame):
    "A game where players never change position, 0 always first, 4 always last"
    
    def play(self, verbose=0):
        """ verbose: 0 no output, 1 only game stats, 2 output for human interaction, 3 full output"""
        self.verbose = verbose
        for round in range(self.max_rounds):

            hand_size = 5 - ((round+self.starting_round) % 5)

            if self.verbose >= 2:
                print(f"\n--- Round {round+1}: {hand_size} card(s) ---")

            if hand_size > 1:
                self.dealHands(hand_size)
                self.makePredictions(round, hand_size)

                # play cards
                for turn in range(hand_size):
                    self.first_in_turn[round, self.player_node.player.id] += 1
                    self.last_in_turn[round, goto_player(
                        self.player_node, (self.player_node.player.id+len(self.players)-1) % len(self.players)).player.id] += 1
                    if self.verbose >= 2:
                        print(f"--- Turn {turn+1} ---")

                    played_cards = self.playCards()
                    self.determineCatches(round, played_cards)
            else:
                visible_cards = []
                for p in self.players:
                    hand = self.deck.draw_hand(1)[0]
                    visible_cards.append(hand)
                    if self.verbose >= 3:
                        print(f"Player {p.id} hand: [{hand.strc()}]")

                self.makePredictions(round, hand_size, visible_cards)
                played_cards = {}
                for p in self.players:
                    played_cards[p.id] = visible_cards[p.id]
                self.determineCatches(round, played_cards)

                if self.verbose == 2:
                    print(f"All cards: [{', '.join([card.strc() for card in visible_cards])}]")

            if self.verbose >= 1:
                print(" ")

            removed_players = self.checkForErrors(round)
            if removed_players:
                if len(self.players) == 1 and self.verbose >= 1:
                    print(f"Winner: {self.players[0].id}")
                if len(self.players) <= 1:
                    if self.verbose >= 1:
                        print("Game Over")
                    break

            # self.current_player = self.current_player.next
            self.deck = Deck()

    def determineCatches(self, round, played_cards):
        ace = False
        for id, card in played_cards.items():
            if card.rank == '1' and card.suit == 'Denari':
                if self.predictions[round, id] > self.catches[round, id]:
                    ace = True
                    self.catches[round, id] += 1
                    # self.player_node = goto_player(self.player_node, id)
                    if self.verbose >= 2:
                        print(f"Player {id} takes")
                else:
                    # lowest possible card
                    played_cards[id] = Card('Bastoni', '1')

        if not ace:
            max = None
            max_idx = None
            for idx, card in played_cards.items():
                if max is None or card > max:
                    max = card
                    max_idx = idx
            self.catches[round, max_idx] += 1
            # self.player_node = goto_player(self.player_node, max_idx)
            if self.verbose >= 2:
                print(f"Player {max_idx} takes")


In [2]:
reps = 10000

for r in range(5):
    print(f"--- Round with {5-r} cards: ---")

    print("With random players:")
    players = [RandomPlayer(0), RandomPlayer(1), RandomPlayer(2), RandomPlayer(3)]
    stat_winners = np.zeros((reps, len(players)))

    for t in range(reps):
        game = PlayersGame(players, r=1, e=-1, s=r)
        game.play(verbose=0)
        stats = game.return_stats()
        stat_winners[t, :] = stats["winners"]

    for i in range(len(players)):
        wins = np.sum(stat_winners[:, i] == 1)
        print(f"{i+1}° Player: Total wins: {wins}")


    print("With heuristic players:")
    players = [CiroPlayer(0), CiroPlayer(1), CiroPlayer(2), CiroPlayer(3)]
    stat_winners = np.zeros((reps, len(players)))

    for t in range(reps):
        game = PlayersGame(players, r=1, e=-1, s=r)
        game.play(verbose=0)
        stats = game.return_stats()
        stat_winners[t, :] = stats["winners"]

    for i in range(len(players)):
        wins = np.sum(stat_winners[:, i] == 1)
        print(f"{i+1}° Player: Total wins: {wins}")

--- Round with 5 cards: ---
With random players:
1° Player: Total wins: 3715
2° Player: Total wins: 3826
3° Player: Total wins: 3832
4° Player: Total wins: 3700
With heuristic players:
1° Player: Total wins: 6180
2° Player: Total wins: 6591
3° Player: Total wins: 6897
4° Player: Total wins: 6671
--- Round with 4 cards: ---
With random players:
1° Player: Total wins: 3993
2° Player: Total wins: 3943
3° Player: Total wins: 3956
4° Player: Total wins: 3897
With heuristic players:
1° Player: Total wins: 6303
2° Player: Total wins: 6905
3° Player: Total wins: 7141
4° Player: Total wins: 6480
--- Round with 3 cards: ---
With random players:
1° Player: Total wins: 4140
2° Player: Total wins: 4363
3° Player: Total wins: 4261
4° Player: Total wins: 4168
With heuristic players:
1° Player: Total wins: 6437
2° Player: Total wins: 7109
3° Player: Total wins: 7553
4° Player: Total wins: 6448
--- Round with 2 cards: ---
With random players:
1° Player: Total wins: 4766
2° Player: Total wins: 4777
3° P

We see that with a heuristic, playing as last doesn't seems favorable.
Random players games are used as controls, without a strategy, going first or last should not change the outcome.

## Define the best theoretical strategy

### for 1 card

The only certain information a player has are the cards he can see.
This means that there is no advantage based on the starting position.
With this only, the best strategy is:
guess 0 if your card is probably not the highest => the highest of the card you see is above the avg. of the cards left (40-the cards of the other players)
guess 1 otherwise

<br>
When all players have optimal strategy, the predicitona of other players become relevant too, especially the one from the last players.
A prediction of 1 means that they belive they have a fair chance of taking => there isn't a card higher that the average.

In [4]:

reps = 10000

for n in range(1, 8):  # n random + 1 Lodo = da 2 a 8 giocatori totali
    print(f"--- With {n+1} players (Lodo last) ---")

    # contatore errori complessivi del Lodo su reps partite da 1 carta
    lodo_total_errors = 0

    for t in range(reps):
        # istanze FRESCHE ogni partita
        players = [RandomPlayer(i) for i in range(n)] + [LodoPlayer(n)]
        game = PlayersGame(players, r=1, e=-1, s=4)  # mano = 1 carta
        game.play(verbose=0)
        stats = game.return_stats()

        # errore del Lodo in questa partita (0 o 1)
        # con r=1 puoi usare stats["errors"][0, n] oppure stats["tot_errors"][n]
        lodo_total_errors += int(stats["tot_errors"][n])

    lodo_correct = reps - lodo_total_errors
    print(f"Lodo correct (no-error) games: {lodo_correct} / {reps} "
          f"({lodo_correct / reps * 100:.2f}%)")



--- With 2 players (Lodo last) ---
Lodo correct (no-error) games: 7469 / 10000 (74.69%)
--- With 3 players (Lodo last) ---
Lodo correct (no-error) games: 7685 / 10000 (76.85%)
--- With 4 players (Lodo last) ---
Lodo correct (no-error) games: 7923 / 10000 (79.23%)
--- With 5 players (Lodo last) ---
Lodo correct (no-error) games: 8323 / 10000 (83.23%)
--- With 6 players (Lodo last) ---
Lodo correct (no-error) games: 8549 / 10000 (85.49%)
--- With 7 players (Lodo last) ---
Lodo correct (no-error) games: 8769 / 10000 (87.69%)
--- With 8 players (Lodo last) ---
Lodo correct (no-error) games: 8883 / 10000 (88.83%)


This strategy consistently wins against a random binary choice even with a high number of players.

### 2 cards